In [1]:
# import necessary libraries and read in data
import altair as alt, pandas as pd, numpy as np
from IPython.display import HTML, display
import vega_datasets as vd

link = "https://raw.githubusercontent.com/apretko11/Visualizations/main/scrubbed.csv"
data = pd.read_csv(link) # this gives us a warning that columns 'duration (seconds)' and 'longitude ' have mixed types

# select which columns to work with and rename columns as needed
cols = ['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)', 'latitude', 'longitude ']
data = data[cols]
data.rename(columns = {'longitude ':'longitude', 'datetime':'IncidentDate', 'duration (seconds)': 'duration'}, inplace = True)

# Convert 'IncidentDate' to datetime
data['IncidentDate'] = pd.to_datetime(data['IncidentDate'], errors='coerce')

# Handle '24:00' by converting it to '00:00' of the next day
data.loc[data['IncidentDate'].dt.hour == 24, 'IncidentDate'] += pd.Timedelta(hours=1)

# Remove the non-numerical rows from 'duration'
data['duration'] = pd.to_numeric(data['duration'], errors='coerce')
narows = data[data['duration'].isna()]
data = data.loc[~data['duration'].isna()]  # Use .loc here

# Remove the non-numerical rows from 'longitude'
data['latitude'] = pd.to_numeric(data['latitude'], errors='coerce')
narows = data[data['latitude'].isna()]
data = data.loc[~data['latitude'].isna()]  # Use .loc here

# Filter data to include only dates since January 1, 1965
data = data[data['IncidentDate'] >= '1965-01-01']

# Extract year and month from 'IncidentDate'
data['Year'] = data['IncidentDate'].dt.year
data['Month'] = data['IncidentDate'].dt.month

# Count of incidents by year
count_by_years = data.groupby('Year').size().reset_index(name='Count')
count_by_both = data.groupby(['Year', 'Month']).size().reset_index(name='Count').sort_values(by=['Year', 'Month'], ascending=True)

# Average duration of incidents by year
average_duration_by_both = data.groupby(['Year', 'Month'])['duration'].mean().reset_index(name='AverageDurationByBoth')
median_duration_by_both = data.groupby(['Year', 'Month'])['duration'].median().reset_index(name='MedianDurationByBoth')

def count_duration_by_time(selected_year):
    # Create Altair chart for count of incidents by year
    count_chart = alt.Chart(count_by_years).mark_bar().encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labels=True)),
        y=alt.Y('Count:Q', title='Count'),
        color=alt.value('blue'),
        tooltip=['Year:N', 'Count:Q']
    ).properties(title='Count of UFO Incidents by Year')

    # Create a bar chart to show the count of UFO incidents by month for the selected year
    count_by_month_chart = alt.Chart(count_by_both).transform_filter(
        alt.datum['Year'] == selected_year
    ).mark_bar().encode(
        x='Month:O',
        y='Count:Q',
        color=alt.value('red'),
        tooltip=['Month:N', 'Count:Q']
    ).properties(title=f'Count of UFO Incidents by Month for {selected_year}')

    # Create a chart to show the median encounter duration for each month
    median_duration_chart = alt.Chart(median_duration_by_both).transform_filter(
        alt.datum['Year'] == selected_year
    ).mark_bar().encode(
        x='Month:O',
        y=alt.Y('MedianDurationByBoth:Q', title='Median Duration in Seconds', axis=alt.Axis(labels=True)),
        tooltip=['Month:N', 'MedianDurationByBoth:Q'],
        color=alt.value('purple')
    ).properties(title=f'Median Encounter Duration by Month for {selected_year}')

    # Create a chart to show the average encounter duration for each month with green color
    average_duration_chart = alt.Chart(average_duration_by_both).transform_filter(
        alt.datum['Year'] == selected_year
    ).mark_bar().encode(
        x='Month:O',
        y=alt.Y('AverageDurationByBoth:Q', title='Average Duration in Seconds', axis=alt.Axis(labels=True)),
        color=alt.value('green'),
        tooltip=['Month:N', 'AverageDurationByBoth:Q']
    ).properties(title=f'Average Encounter Duration by Month for {selected_year}')

    final_output = alt.vconcat(count_chart, (count_by_month_chart | median_duration_chart | average_duration_chart))

    return (final_output)

def create_duration_charts(start_year, stop_year):

    data_filtered = data[(data['Year'] >= start_year) & (data['Year'] <= stop_year)]
    average_duration_by_year = data_filtered.groupby('Year')['duration'].mean().reset_index()
    average_duration_by_year = average_duration_by_year.rename(columns={'duration': 'AverageDurationByYear'})

    # Calculate median duration by year
    median_duration_by_year = data_filtered.groupby('Year')['duration'].median().reset_index()
    median_duration_by_year = median_duration_by_year.rename(columns={'duration': 'MedianDurationByYear'})

    # Create average duration chart
    average_duration_by_year_chart = alt.Chart(average_duration_by_year).mark_line().encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labels=True)),
        y=alt.Y('AverageDurationByYear:Q', title='Average Duration in Seconds', axis=alt.Axis(labels=True))
    ).properties(
        title=f'Average Duration of Incidents by Year (in Seconds): {start_year} to {stop_year}'
    )

    # Create median duration chart
    median_duration_by_year_chart = alt.Chart(median_duration_by_year).mark_line().encode(
        x=alt.X('Year:O', title='Year', axis=alt.Axis(labels=True)),
        y=alt.Y('MedianDurationByYear:Q', title='Median Duration in Seconds', axis=alt.Axis(labels=True))
    ).properties(
        title=f'Median Duration of Incidents by Year (in Seconds): {start_year} to {stop_year}'
    )

    return (average_duration_by_year_chart | median_duration_by_year_chart)

def create_ufo_chart_states(selected_year):

  # Select year data, country IS NOT NULL, binary domestic/international category
  data['Category'] = data['country'].apply(lambda x: 'Domestic' if x == 'us' else 'International')
  filtered_data = data[data['Year'] == selected_year].dropna(subset=['country'])

  if len(filtered_data) > 5000:
    filtered_data = filtered_data.sample(n=5000, random_state=42)  # Adjust the random_state as needed

  non_us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] != 'us')]
  us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] == 'us')]

  # Create a wider map chart
  map_chart = alt.Chart(filtered_data).mark_circle(size=10, opacity=0.6).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('Category:N', scale=alt.Scale(range=['blue', 'green'])),  # Color based on the category
    tooltip=['city:N', 'state:N', 'country:N'],
  ).project('albersUsa').properties(
    width=800,  # Adjust the width of the map chart
    height=400,
    title=f'Map of UFO Sightings in {selected_year}'
    )

  # Load state boundaries data
  states = alt.topo_feature(vd.data.us_10m.url, 'states')

  # Add state boundaries to the map
  background = alt.Chart(states).mark_geoshape(
      fill=None,
      stroke='lightgray',  # Light gray stroke for state boundaries
  ).project('albersUsa')

  # Compute counts & percentages for each state then sort
  state_counts = us_data['state'].value_counts().reset_index()
  state_counts.columns = ['state', 'Count']
  total_count = state_counts['Count'].sum()
  state_counts['Percentage'] = (state_counts['Count'] / total_count) * 100
  state_counts = state_counts.sort_values(by='Count', ascending=False)
  top_states = state_counts.head(25)

  # Create a bar chart for the top 10 states
  bar_chart = alt.Chart(top_states).mark_bar().encode(
      x=alt.X('state:N', sort='-y'),  # Sort the states by count in descending order
      y='Count:Q',
      tooltip=['state:N', 'Count:Q'],
  ).properties(
      width=800,
      title=f'Top 25 States with the Most UFO Sightings in {selected_year}'
  )

  return ((background + map_chart)) & bar_chart

def create_ufo_chart_international(start_year, stop_year, selected_year):

  # Select year data, country IS NOT NULL, binary domestic/international category
  data['Category'] = data['country'].apply(lambda x: 'Domestic' if x == 'us' else 'International')
  filtered_data = data[data['Year'] == selected_year].dropna(subset=['country'])

  if len(filtered_data) > 5000:
    filtered_data = filtered_data.sample(n=5000, random_state=42)  # Adjust the random_state as needed

  non_us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] != 'us')]
  us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] == 'us')]

  # Calculate the country percentages for the filtered data
  country_data_non_us = non_us_data.groupby('country').size().reset_index(name='Count')
  country_data_non_us['Percentage'] = (country_data_non_us['Count'] / country_data_non_us['Count'].sum() * 100)

  # Create a pie chart for non-US domestic incidents with color determined by "country"
  pie_chart_non_us = alt.Chart(country_data_non_us).mark_arc().encode(
      color='country:N',
      tooltip=['country:N', 'Percentage:Q'],
      theta='Percentage:Q'
  ).properties(
      width=800,  # Adjust the width as needed
      height=200,  # Adjust the height as needed
      title=f'{selected_year} UFO Sightings Percentage (%) Breakdown by Country'
  )

  # Calculate the percentage split of Domestic and International sightings for each year
  yearly_category_counts = data[(data['Year'] >= start_year) & (data['Year'] <= stop_year)].groupby(['Year', 'Category']).size().unstack(fill_value=0)
  yearly_category_counts['Total'] = yearly_category_counts.sum(axis=1)
  yearly_category_counts['Domestic_Percentage'] = (yearly_category_counts['Domestic'] / yearly_category_counts['Total']) * 100
  yearly_category_counts['International_Percentage'] = (yearly_category_counts['International'] / yearly_category_counts['Total']) * 100

  # Reset the index for the DataFrame
  yearly_category_counts = yearly_category_counts.reset_index()

  # Create a stacked bar chart
  stacked_bar_chart = alt.Chart(yearly_category_counts).mark_bar().encode(
      x=alt.X('Year:O', title='Year', axis=alt.Axis(labels=True)),
      y=alt.Y('Domestic_Percentage:Q', title='Percentage (%) Domestic'),
      tooltip=['Year:N', 'Domestic_Percentage:Q', 'International_Percentage:Q']
  ).properties(
      width=800,  # Adjust the width as needed
      height=300,  # Adjust the height as needed
      title=f'Domestic vs. International Split From {start_year} To {stop_year}'
  )

  # Display the combined charts
  return (stacked_bar_chart | pie_chart_non_us)

def distribution_over_time(selected_year):

    # Select year data, country IS NOT NULL, binary domestic/international category
    data['Category'] = data['country'].apply(lambda x: 'Domestic' if x == 'us' else 'International')
    filtered_data = data[data['Year'] == selected_year].dropna(subset=['country'])
    us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] == 'us')]

    # Filter out extremely long durations (outliers)
    max_duration = 10000  # Adjust as needed
    us_data = us_data[us_data['duration'] <= max_duration]

    # sample to get down below 5000
    if len(us_data) > 5000:
        us_data = us_data.sample(n=5000, random_state=42)

    # Create a histogram without density on the left
    histogram = alt.Chart(us_data).mark_bar().encode(
        alt.X('duration:Q', bin=alt.Bin(maxbins=50), title='Duration in Seconds'),
        alt.Y('count()', title='Frequency')
    ).properties(
        width=800,  # Adjust the width of the left chart
        title=f'Duration Histogram for Year {selected_year}'
    )

    # Create a chart with density on the right
    density_chart = alt.Chart(us_data).transform_density(
        density='duration',
        as_=['duration', 'density']
    ).mark_area(opacity=0.5).encode(
        alt.X('duration:Q', title='Duration in Seconds'),
        alt.Y('density:Q', title='Density')
    ).properties(
        width=800,  # Adjust the width of the right chart
        title=f'Density Plot for Year {selected_year}'
    )

    # Concatenate the two charts horizontally
    combined_chart = histogram | density_chart

    return combined_chart

def full_distribution():
    # Select data where the country IS NOT NULL
    data['Category'] = data['country'].apply(lambda x: 'Domestic' if x == 'us' else 'International')
    filtered_data = data.dropna(subset=['country'])
    us_data = filtered_data[(filtered_data['country'].notnull()) & (filtered_data['country'] == 'us')]

    # Filter out extremely long durations (outliers)
    max_duration = 10000  # Adjust as needed
    us_data = us_data[us_data['duration'] <= max_duration]

    if len(us_data) > 5000:
        us_data = us_data.sample(n=5000, random_state=42)

    # Create a histogram without density
    histogram = alt.Chart(us_data).mark_bar().encode(
        alt.X('duration:Q', bin=alt.Bin(maxbins=50), title='Duration in Seconds'),
        alt.Y('count()', title='Frequency')
    ).properties(
        width=800,
        title='Full Dataset Duration Histogram'
    )

    # Create a chart with density
    density_chart = alt.Chart(us_data).transform_density(
        density='duration',
        as_=['duration', 'density']
    ).mark_area(opacity=0.5).encode(
        alt.X('duration:Q', title='Duration in Seconds'),
        alt.Y('density:Q', title='Density')
    ).properties(
        width=800,
        title='Full Dataset Density Plot'
    )

    # Concatenate the two charts vertically
    combined_chart = histogram | density_chart

    return combined_chart

def create_distribution_charts(selected_year):

    distribution_over_time_chart = distribution_over_time(selected_year)
    full_distribution_chart = full_distribution()

    return full_distribution_chart & distribution_over_time_chart

#########################################################

def save_chart_as_html(chart, file_name):
    chart.save(file_name)

def show_it_all(first_year, final_year, focus_year):

    # Check if selected_year is within the specified range
    if not (first_year <= final_year):
        raise ValueError("selected_year must be within the range of start_year and stop_year.")

    if first_year not in data['Year'].unique():
        raise ValueError(f"first_year {first_year} must exist in the data['Year'] column.")

    if final_year not in data['Year'].unique():
        raise ValueError(f"final_year {final_year} must exist in the data['Year'] column.")

    if focus_year not in data['Year'].unique():
        raise ValueError(f"focus_year {focus_year} must exist in the data['Year'] column.")

    count_duration_chart = count_duration_by_time(focus_year)
    display(count_duration_chart)

    duration_charts = create_duration_charts(first_year, final_year)
    display(duration_charts)

    ufo_chart_states = create_ufo_chart_states(focus_year)
    display(ufo_chart_states)

    ufo_chart_international = create_ufo_chart_international(first_year, final_year, focus_year)
    display(ufo_chart_international)

    distro_charts = create_distribution_charts(focus_year)
    display(distro_charts)

def run_it_all(first_year, final_year, focus_year):

    # Check if selected_year is within the specified range
    if not (first_year <= final_year):
        raise ValueError("selected_year must be within the range of start_year and stop_year.")

    if first_year not in data['Year'].unique():
        raise ValueError(f"first_year {first_year} must exist in the data['Year'] column.")

    if final_year not in data['Year'].unique():
        raise ValueError(f"final_year {final_year} must exist in the data['Year'] column.")

    if focus_year not in data['Year'].unique():
        raise ValueError(f"focus_year {focus_year} must exist in the data['Year'] column.")

    count_duration_chart = count_duration_by_time(focus_year)
    save_chart_as_html(count_duration_chart, "count_duration_chart.html")

    duration_charts = create_duration_charts(first_year, final_year)
    save_chart_as_html(duration_charts, "duration_charts.html")

    ufo_chart_states = create_ufo_chart_states(focus_year)
    save_chart_as_html(ufo_chart_states, "ufo_chart_states.html")

    ufo_chart_international = create_ufo_chart_international(first_year, final_year, focus_year)
    save_chart_as_html(ufo_chart_international, "ufo_chart_international.html")

    distro_charts = create_distribution_charts(focus_year)
    save_chart_as_html(distro_charts, "distro_charts.html")

def get_user_input():
    first_year = int(input("Enter the first year (1965-2014): "))
    final_year = int(input("Enter the final year (1965-2014): "))
    focus_year = int(input("Enter the focus year (1965-2014): "))
    return first_year, final_year, focus_year

#################################### Uncomment by holding ctrl then pressing /

# if __name__ == "__main":

#     print("Remember this program has data from 1965 through 2014 - please choose your dates appropriately!")

#     all_outputs = input("Do you want all the outputs? (Yes or No): ").strip().lower()

#     if all_outputs == "yes":
#         first_year, final_year, focus_year = get_user_input()
#         run_it_all(first_year, final_year, focus_year)
#     elif all_outputs == "no":
#         output_choice = input("Which output do you want (count, duration, states, international, or distribution)? ").strip().lower()
#         if output_choice == "count":
#             first_year, final_year, focus_year = get_user_input()
#             count_duration_chart = count_duration_by_time(focus_year)
#             save_chart_as_html(count_duration_chart, "count_duration_chart.html")
#         elif output_choice == "duration":
#             first_year, final_year, focus_year = get_user_input()
#             duration_charts = create_duration_charts(first_year, final_year)
#             save_chart_as_html(duration_charts, "duration_charts.html")
#         elif output_choice == "states":
#             focus_year = int(input("Enter the focus year: "))
#             ufo_chart_states = create_ufo_chart_states(focus_year)
#             save_chart_as_html(ufo_chart_states, "ufo_chart_states.html")
#         elif output_choice == "international":
#             first_year, final_year, focus_year = get_user_input()
#             ufo_chart_international = create_ufo_chart_international(first_year, final_year, focus_year)
#             save_chart_as_html(ufo_chart_international, "ufo_chart_international.html")
#         elif output_choice == "distribution":
#             focus_year = int(input("Enter the focus year: "))
#             distro_charts = create_distribution_charts(focus_year)
#             save_chart_as_html(distro_charts, "distro_charts.html")
#         else:
#             print("Invalid output choice. Please choose from count, duration, states, international, or distribution.")
#     else:
#         print("Invalid choice. Please answer 'Yes' or 'No'.")

<ipython-input-1-a38076b2fbdf>:7: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(link) # this gives us a warning that columns 'duration (seconds)' and 'longitude ' have mixed types
<ipython-input-1-a38076b2fbdf>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['latitude'] = pd.to_numeric(data['latitude'], errors='coerce')


In [4]:
# change the start_year, stop_year, and focus_year in the function below
show_it_all(1965,2014,1973)

alt.VConcatChart(...)

alt.HConcatChart(...)

alt.VConcatChart(...)

alt.HConcatChart(...)

alt.VConcatChart(...)